In [5]:
# -------------------------------------------------------------
#
# scipy's "linkage()" function can't handle our data sizes
#
# ------------------------------------------------------------
%matplotlib inline


import time
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import AgglomerativeClustering, ward_tree
from sklearn.neighbors import kneighbors_graph
from scipy.cluster.hierarchy import dendrogram
from sklearn.utils.validation import check_memory

K = 100
N = 20
FILE = 'data/run-pink.npy'

class AgglomerativeClusterer():
    def __init__(self, filename=FILE, N=N):
        self.filename = filename
        self.X = np.load(filename)
        print('X', self.X.shape)
        self.idx = np.arange(self.X.shape[0])
        self.link_type = 'ward'
        self.affinity = 'euclidean'
        self.N = N
        self.event_labels  = {i:i for i in range(self.N)}
        self.connectivity = None
    
    def to_range(self, l, cmap):
        total = cmap.N
        idx = int((l[-1] / self.N) * total)
        return cmap(idx)

    def boundaries(self, v):
        bounds = [0] + list(np.where(v[:-1] != v[1:])[0] + 1) + [len(v)-1]
        return [(bounds[i], bounds[i+1], v[bounds[i]]) for i in range(len(bounds)-1)]
    
    def remap_labels(self, labels):
        map_keys = np.argsort(np.unique(labels, return_counts=True)[1]).reshape(self.N, 1)[::-1]
        vals, idx = np.where(labels == map_keys)
        labels[idx] = vals
        return labels
    
    def compute_connectivity(self, k=100):
        t0 = time.time()
        graph = kneighbors_graph(self.X, k, include_self=False, n_jobs=-1)
        elapsed_time = time.time() - t0
        self.connectivity = graph
        print('setting connectivity:', graph.shape, graph.nnz)
        return elapsed_time

    def run_clustering(self, n=None):
        n = n or self.N
        model = AgglomerativeClustering(linkage=self.link_type, affinity=self.affinity,
                                        n_clusters=n, connectivity=self.connectivity,
                                        memory='HAC_cache')
        start = time.time()
        model.fit(self.X)
        end = time.time()
        total_time = end - start
        
        model_labels = model.labels_
        labels = self.remap_labels(model_labels)
        return (total_time, model, labels, np.unique(labels, return_counts=True)[1])
    
    def distances(self):
        mem = check_memory('HAC_cache')
        func = mem.cache(ward_tree)
        start = time.time()
        children, _, __, parents, dists = func(self.X, self.connectivity, n_clusters=self.N, return_distance=True)
        end = time.time()
        total_time = end - start
        return total_time, children, parents, dists
        

ac = AgglomerativeClusterer(FILE, N)
ctime = ac.compute_connectivity(K)
print('connectivity time: ', ctime)
t, mdl, lbls, counts = ac.run_clustering()
print('clustering time: ', t)

print('labels:', lbls.shape)
print('counts:', counts)

print('# leaves:', mdl.n_leaves_)
print('# components:', mdl.n_components_)

print('children:', mdl.children_.shape)
print(mdl.get_params(True))
print('saving child tree: "data/pb-hac.csv"')
np.savetxt('data/pb-hac.csv', mdl.children_, delimiter=', ', fmt='%i')

Zt, Zc, Zp, Zd = ac.distances()
print('Z time:', Zt)
print('children:', Zc.shape)
print('parents:', Zp.shape)
print('distances:', Zd.shape)


LABEL = np.array([26520 / 10, 35769 / 10])

def closest_node(node, nodes):
    nodes = np.asarray(nodes)
    dist_2 = np.sum((nodes - node)**2, axis=1)
    return np.argmin(dist_2)

closest = closest_node(LABEL, mdl.children_)
c = mdl.children_[closest]
print('closest:', closest, c)

region = lbls[c[0]:c[1]]
xl = np.unique(region, return_counts=True)
print('labels of closest:', region.shape, xl)
print(region)

X (63039, 13)
setting connectivity: (63039, 63039) 6303900
connectivity time:  5.715408563613892
clustering time:  84.74975895881653
labels: (63039,)
counts: [13248 11421  7516  7369  6647  3769  3657  3436  1866   771   732   527
   471   451   333   306   257   102    82    78]
# leaves: 63039
# components: 1
children: (63038, 2)
{'affinity': 'euclidean', 'compute_full_tree': 'auto', 'connectivity': <63039x63039 sparse matrix of type '<class 'numpy.float64'>'
	with 6303900 stored elements in Compressed Sparse Row format>, 'linkage': 'ward', 'memory': 'HAC_cache', 'n_clusters': 20, 'pooling_func': <function mean at 0x7fc75c047e18>}
saving child tree: "data/pb-hac.csv"
Z time: 88.01886343955994
children: (63019, 2)
parents: (126058,)
distances: (63019,)
closest: 38765 [2830 3728]
labels of closest: (898,) (array([0, 1, 2, 3, 4, 5, 6, 7, 9]), array([170, 132, 115, 150, 116,  36,  75,  22,  82]))
[4 4 4 4 4 5 5 5 5 5 5 7 7 7 7 7 7 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0
 0 6 6 0 0 0 0 0 

closest: 66163 [2702 3552]
labels of closest: (850,) (array([ 0,  1,  2,  3,  4,  5,  6,  7,  8, 10, 16, 17, 19]), array([119, 191,  57,  37, 124,  16, 117,  20,  36, 106,  18,   7,   2]))
[19  8  3 10  2  2  2  2  3  3  3  3  3  1  3  0  3  0  0  0  0  7  7  7
  7  1  1  1  1  1  1 10  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  4
  2  2  2  1  1  1  1  1  1  1  1  0  0  0  0  0  0  0  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  5  1  5  5  5  5  5  5  1  5  5  5  5  5  5  5  5  5  8  8  8  8
  8  8  8  8  8  8  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4
  4  4  4  4  4  4  4  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2
  2  2  2  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4
  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4
  4  4  4  1  1  1  1  4  4  4  4  1  1  1  4  4  4  4  4  4  4  4  4  4
  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4
  4  4  